In [ ]:
%pip install -U sentence-transformers
%pip install catboost
%pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6484a4ac5476324a680704a65f2aa05533c09baaddd5c6e36f04f570832d796c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.4 MB/s eta 0:00:00


In [ ]:
%pip install transformers
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install datasets
%pip install -U accelerate

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import math
import pandas as pd
import requests
from urllib.parse import urlencode
import os
import numpy as np
from catboost import CatBoostRanker, Pool
from functools import partial
import json
import pyarrow.parquet as pq
import time

# Загрузка данных

In [ ]:
start_time = time.time()

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
!pip install fastparquet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Формирование базы векторов

In [ ]:
candidates = pd.read_parquet(
    '/content/drive/MyDrive/videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

In [ ]:
test_queries=pd.read_csv('/content/drive/MyDrive/test_dataset_submission_queries.csv')

In [ ]:
candidates = candidates.sample(n=7_000_000, replace=False, random_state=seed)

In [ ]:
corpus = candidates['video_title'].apply(lambda x: x.lower()).values
video_ids = candidates['video_id'].values
del candidates

In [ ]:
video_ids

array(['video_24972264', 'video_22470853', 'video_22923068', ...,
       'video_26140712', 'video_9455727', 'video_1275091'], dtype=object)

In [ ]:
st_model = SentenceTransformer(
    '/content/drive/MyDrive/RuTubeLaBSE',
    device='cuda'
)
# st_model = SentenceTransformer(
#     'cointegrated/LaBSE-en-ru',
#     device='cuda'
# )

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/RuTubeLaBSE and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# когда прогоните один раз у вас на диске уже будет сохранен faiss индекс
# можно поставить значение True, чтобы сэкономить время на формирование индекса
use_formed_index = False

In [ ]:
d = 768
if not use_formed_index:
    cpu_index = faiss.IndexFlatL2(d)
    cpu_index.is_trained, cpu_index.ntotal

In [ ]:
# если уже есть файл ind2videoid для вашего faiss индекса - True
use_formed_id_mapping = True

if not use_formed_id_mapping:
    ind2videoid = {ind: video_id for ind, video_id in enumerate(video_ids)}
    with open('ind2videoid.json', 'w+') as f:
        json.dump(ind2videoid, f, indent=4)
else:
    with open('ind2videoid.json', 'r') as f:
        ind2videoid = json.load(f)

In [ ]:
batch_size = 1_000_000
num_batches = math.ceil(len(corpus) / batch_size)

In [ ]:
if not use_formed_index:
    try:
        for i in range(num_batches):
            # формируем батч
            start, end = i * batch_size, (i + 1) * batch_size
            corpus_batch = corpus[start:end]

            # считаем вектора для всех предложений в батче
            embeddings = st_model.encode(
                corpus_batch,
                batch_size=6000,
                show_progress_bar=True
            )

            # добавляем новые батч векторов в индекс и сохраняем его
            cpu_index.add(embeddings)
            write_index(cpu_index, 'candidates.index')

            print(f'batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}')

            # чистим ОЗУ
            del embeddings
    except KeyboardInterrupt:
        print('Остановлено пользователем')
        try:
            del embeddings
        except:
            pass

Batches:   0%|          | 0/167 [00:00<?, ?it/s]

batch: 1 / 7, vectors: 1000000


Batches:   0%|          | 0/167 [00:00<?, ?it/s]

batch: 2 / 7, vectors: 2000000


Batches:   0%|          | 0/167 [00:00<?, ?it/s]

In [ ]:
# !cp candidates.index /content/drive/MyDrive/dataset/candidates.index

In [ ]:
# чистим ОЗУ
if not use_formed_index:
    del cpu_index

In [ ]:
# !cp generated_candidates.parquet /content/drive/MyDrive/dataset/generated_candidates.parquet

# Подбор кандидатов по базе векторов

In [ ]:
automarkup = pd.read_parquet(
    '/content/drive/MyDrive/dataset/automarkup.parquet',
    engine='fastparquet'
)

In [ ]:
automarkup = automarkup[~automarkup['query'].isna()]
automarkup['query'] = automarkup['query'].apply(lambda x: x.lower())

In [ ]:

queries = test_queries
query2ind = {q: i for i, q in enumerate(queries)}

In [ ]:
# когда прогоните один раз у вас на диске кандиды уже будут сохранены
# можете поставить значение True, чтобы сэкономить время
use_formed_candidates = False

In [ ]:
qembeddings = st_model.encode(
    queries,
    batch_size=20000,
    show_progress_bar=True
)

search_cpu_index = read_index('/content/candidates.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(True, 7000000)

In [ ]:
generated_cand_name = 'generated_candidates.parquet'

if not use_formed_candidates:
    topk = 5
    distance, faiss_ind = search_cpu_index.search(qembeddings, topk)

    # print(distance)
    generated_cand = {
        'query': [],
        'video_id': []
    }

    for i, q in enumerate(queries):
        vids = faiss_ind[i]
        dist = distance[i]
        try:
          generated_cand['video_id'] += [ind2videoid[v] for v in vids]
          generated_cand['query'] += [q] * len(vids)
        except KeyError:
          ...

    generated_cand = pd.DataFrame(generated_cand)

    generated_cand.to_csv('test.csv')
else:
    generated_cand = pd.read_parquet(
        generated_cand_name,
        engine='fastparquet'
    )

In [ ]:
!cp generated_candidates.parquet /content/drive/MyDrive/dataset/generated_candidates.parquet

cp: cannot stat 'generated_candidates.parquet': No such file or directory


# Формирование таргета по авторазметке

In [ ]:
# generated_cand = pd.read_parquet(
#         '/content/drive/MyDrive/dataset/generated_candidates.parquet',
#         engine='fastparquet'
#     )

In [ ]:
automarkup['target'] = [1] * automarkup.shape[0]
candidates_with_target = generated_cand.merge(
    automarkup[['query', 'video_id', 'target']],
    how='left',
    left_on=['query', 'video_id'],
    right_on=['query', 'video_id']
)
candidates_with_target['target'] = candidates_with_target['target'].fillna(0)

In [ ]:
# очищаем ОЗУ
del generated_cand
del search_cpu_index
del automarkup
del st_model
del qembeddings

In [ ]:
# можно не делать

features_parquet = pq.ParquetFile('/content/drive/MyDrive/dataset/features.parquet')
features, filter_date = None, '2023-05-02'

for batch in features_parquet.iter_batches():
    tmp = batch.to_pandas()
    if features is None:
        features = tmp[tmp['report_date'] == filter_date]
    else:
        features = pd.concat([
            features,
            tmp[tmp['report_date'] == filter_date]
        ], axis=0)

# для baseline выбросим категориальные признаки и datetime признаки
# в своем решении вы сами можете решить использовать их или нет
features = features.drop(
    [
        'v_channel_reg_datetime',
        'v_channel_type',
        'v_category',
        'v_pub_datetime'
    ],
    axis=1
)

In [ ]:
# features
features.to_parquet(
    'featuresNew',
    engine='fastparquet'
)

In [ ]:
!cp featuresNew /content/drive/MyDrive/dataset/featuresNew.parquet

# Формирование датасета с признаками

In [ ]:
features = pd.read_parquet(
    '/content/drive/MyDrive/dataset/featuresNew.parquet',
    engine='fastparquet'
)

In [ ]:
full_df = candidates_with_target.merge(
    features,
    how='inner',
    left_on='video_id',
    right_on='video_id'
)
full_df = full_df.drop('report_date', axis=1)
full_df = full_df.drop_duplicates()

In [ ]:
full_df = full_df.drop(['similarity_score'], axis=1)

In [ ]:
full_df

,query,video_id,target,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,...,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days,group_id
0,битва сильнейших,video_6006905,0.0,0,11,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,431
1,битва сильнейших,video_6006905,0.0,0,11,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,432
2,битва сильнейших 3 выпуск,video_6006905,0.0,0,11,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,494
6,битва сильнейших 13 выпуск,video_6006905,0.0,0,11,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,446
7,битва сильнейших 1 выпуск,video_6006905,0.0,0,11,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374033,«пять минут тишины». 2сезон. 3 серия,video_1960282,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,263
374034,«пять минут тишины». 2сезон. 3 серия,video_1544272,0.0,0,3244,122,40,8,0,0,...,0.0,0.0,0.672131,0.0,0.0,0.0,0.0,0.5,0.0,263
374035,«пять минут тишины». 2сезон. 3 серия,video_12735187,0.0,0,5,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,263
374036,«пять минут тишины». 2сезон. 3 серия,video_25719148,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,263


In [ ]:
# full_df=full_df.merge(generated_cand,how='inner',on='video_id')

In [ ]:
# full_df = full_df.drop('similarity_score', axis=1)

In [ ]:
# full_df['similarity_score']

In [ ]:
groups_to_drop = []
full_df['group_id'] = full_df.groupby(['query']).ngroup()
for group in full_df['group_id'].unique():
    part_df = full_df[full_df['group_id'] == group]
    target_sum = part_df['target'].values.sum()
    if target_sum <= 0:
        groups_to_drop += [group]
full_df = full_df[~full_df['group_id'].isin(groups_to_drop)]

In [ ]:
groups = pd.Series(full_df['group_id'].unique())
permutation = groups.sample(frac=1, random_state=seed)
train_groups, val_groups, test_groups = np.split(
    permutation,
    [int(0.75 * len(permutation)), int(0.90 * len(permutation))]
)

In [ ]:
train_df = full_df[full_df['group_id'].isin(train_groups)]
val_df = full_df[full_df['group_id'].isin(val_groups)]
test_df = full_df[full_df['group_id'].isin(test_groups)]

In [ ]:
train_df = train_df.sort_values('group_id')
val_df = val_df.sort_values('group_id')
test_df = test_df.sort_values('group_id')

In [ ]:
metainfo_columns = ['query', 'video_id', 'target', 'group_id']

X_train = train_df.drop(metainfo_columns, axis=1)
y_train, g_train = train_df['target'], train_df['group_id']

X_val = val_df.drop(metainfo_columns, axis=1)
y_val, g_val = val_df['target'], val_df['group_id']

X_test = test_df.drop(metainfo_columns, axis=1)
y_test, g_test = test_df['target'], test_df['group_id']

In [ ]:
train = Pool(
    data=X_train.values,
    label=y_train.values,
    group_id=g_train.values,
    feature_names=X_train.columns.to_list()
)

val = Pool(
    data=X_val.values,
    label=y_val.values,
    group_id=g_val.values,
    feature_names=X_val.columns.to_list()
)

test = Pool(
    data=X_test.values,
    label=y_test.values,
    group_id=g_test.values,
    feature_names=X_test.columns.to_list()
)

# Обучение модели

In [ ]:
task_type = 'GPU'
metric_period = 250

parameters = {
    'task_type': task_type,
    'verbose': False,
    'random_seed': seed,
    'loss_function': 'QueryRMSE',
    'learning_rate': 0.001,
    'l2_leaf_reg': 30,
    'iterations': 4000,
    'max_depth': 3,
}


In [ ]:
model = CatBoostRanker(**parameters)
model = model.fit(
    train,
    eval_set=val,
    plot=True,
    use_best_model=True,
    metric_period=metric_period
)
model.save_model('rankerRutubeModelwithoutSimScore.ckpt')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
!cp rankerRutubeModelwithoutSimScore.ckpt /content/drive/MyDrive/rankerRutubeModelwithoutSimScore.ckpt

# Измерение метрик

In [ ]:
def _metrics_at(at, model, pool, metric='NDCG'):
    metric = metric + f':top={at}'
    eval_metrics = model.eval_metrics(pool, metrics=[metric])
    best_metrics = {}
    for key in eval_metrics.keys():
        best_metrics[key] = eval_metrics[key][model.best_iteration_]
    return best_metrics


metrics_train_at = partial(
    _metrics_at,
    model=model,
    pool=train
)

metrics_val_at = partial(
    _metrics_at,
    model=model,
    pool=val
)

metrics_test_at = partial(
    _metrics_at,
    model=model,
    pool=test
)

In [ ]:
metrics_train_at(1), metrics_train_at(5)
# ({'NDCG:top=1;type=Base': 0.6324503311258278},
#  {'NDCG:top=5;type=Base': 0.6927891290798337})

TypeError: ignored

In [ ]:
metrics_val_at(1), metrics_val_at(5)
# ({'NDCG:top=1;type=Base': 0.6324503311258278},
#  {'NDCG:top=5;type=Base': 0.6927891290798337})

({'NDCG:top=1;type=Base': 0.4485981308411215},
 {'NDCG:top=5;type=Base': 0.5302211748940849})

In [ ]:
metrics_test_at(1), metrics_test_at(5)
# ({'NDCG:top=1;type=Base': 0.6324503311258278},
#  {'NDCG:top=5;type=Base': 0.6927891290798337})

({'NDCG:top=1;type=Base': 0.4166666666666667},
 {'NDCG:top=5;type=Base': 0.5584137127219672})

In [ ]:
importances = model.get_feature_importance(train)

In [ ]:
# очищаем ОЗУ
del model
del full_df
del train_df, val_df, test_df
del train, val, test
del X_train, y_train, g_train
del X_val, y_val, g_val
del X_test, y_test, g_test

In [ ]:
end_time = time.time()
total_time = (end_time - start_time) / 60
print(f'Общее время работы baseline: {int(total_time)} минут')

Общее время работы baseline: 97 минут


# Проверка модели

In [ ]:
task_type = 'GPU'
metric_period = 250

parameters = {
    'task_type': task_type,
    'verbose': False,
    'random_seed': seed,
    'loss_function': 'QueryRMSE',
    'learning_rate': 0.001,
    'l2_leaf_reg': 30,
    'iterations': 4000,
    'max_depth': 3,
}

In [ ]:
from catboost import Pool, cv
# Initialize data
train_data = Pool(data=X_train, label=y_train, group_id=g_train.values)
# Specify the parameter grid
params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3],
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],}
# Run the grid search
tuned_model = CatBoostRanker().grid_search(params,
                                           train_data,
                                           plot=True,
                                           refit=True,
                                           # refit best model on all data
                                           partition_random_seed=42)
print("Best model parameters: ", tuned_model["params"])

In [ ]:
model = CatBoostRanker(**parameters)
# model.save_model('rankerRutubeModel.ckpt')
model.load_model('/content/drive/MyDrive/rankerRutubeModelwithoutSimScore.ckpt')

In [ ]:
model.predict()